## Dự đoán Giá Sản Phẩm

### Tuần 2 Ngày 1

Giới thiệu về LoRA và QLoRA



## Một lưu ý về 2 mẹo quan trọng khi sử dụng Colab:

**Mẹo 1:**

Phía trên cùng của mỗi colab đều có một số lệnh pip install. Bạn có thể gặp phải lỗi từ pip khi chạy các lệnh này, ví dụ như:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

Các lỗi tương thích pip này có thể bỏ qua an toàn; và mặc dù bạn có thể muốn sửa chúng bằng cách thay đổi số phiên bản, nhưng điều đó thực sự sẽ gây ra những vấn đề nghiêm trọng!

**Mẹo 2:**

Trong quá trình chạy Colab, bạn có thể gặp lỗi như sau:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

Đây là một thông báo lỗi rất dễ gây hiểu nhầm! Đừng cố thay đổi phiên bản của các gói...

Thực chất, lỗi này xảy ra vì Google đã thay đổi runtime của bạn, có thể do Google Colab quá tải. Cách giải quyết là:

1. Vào menu Kernel >> Disconnect and delete runtime
2. Tải lại colab từ đầu và vào menu Edit >> Clear All Outputs
3. Kết nối lại với T4 mới bằng nút ở góc trên bên phải
4. Chọn "View resources" từ menu trên cùng bên phải để xác nhận bạn đã có GPU
5. Chạy lại tất cả các cell trong colab, bắt đầu từ lệnh pip install ở trên cùng

Và mọi thứ sẽ hoạt động tốt – nếu không, hãy hỏi mình nhé!

In [ ]:
# pip installs

!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"ed-donner/pricer-2024-09-13_13.04.39"

# Hyperparameters for QLoRA Fine-Tuning

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

### Đăng nhập vào HuggingFace

Nếu bạn chưa có tài khoản HuggingFace, hãy truy cập https://huggingface.co để đăng ký và tạo một token.

Sau đó chọn mục Secrets cho Notebook này bằng cách nhấn vào biểu tượng hình chìa khóa ở bên trái, và thêm một secret mới tên là `HF_TOKEN` với giá trị là token của bạn.

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

## Thử nghiệm các phương pháp Lượng tử hóa khác nhau



In [ ]:
# Load the Base Model without quantization

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## Khởi động lại phiên làm việc của bạn!

Để tải mô hình tiếp theo và xóa bộ nhớ đệm của mô hình trước đó, bạn cần vào Runtime >> Restart session và chạy lại các cell ban đầu (cài đặt, import và đăng nhập HuggingFace).

Việc này giúp làm sạch GPU.

In [ ]:
# Load the Base Model using 8 bit

quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## Khởi động lại phiên làm việc của bạn!

Để tải mô hình tiếp theo và xóa bộ nhớ đệm của mô hình trước đó, bạn cần vào Runtime >> Restart session và chạy lại các cell ban đầu (import và đăng nhập HuggingFace).

Việc này giúp làm sạch GPU.

In [ ]:
# Load the Tokenizer and the Base Model using 4 bit

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
base_model

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

In [ ]:
print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
fine_tuned_model

In [ ]:
# Each of the Target Modules has 2 LoRA Adaptor matrices, called lora_A and lora_B
# These are designed so that weights can be adapted by adding alpha * lora_A * lora_B
# Let's count the number of weights using their dimensions:

# See the matrix dimensions above
lora_q_proj = 4096 * 32 + 4096 * 32
lora_k_proj = 4096 * 32 + 1024 * 32
lora_v_proj = 4096 * 32 + 1024 * 32
lora_o_proj = 4096 * 32 + 4096 * 32

# Each layer comes to
lora_layer = lora_q_proj + lora_k_proj + lora_v_proj + lora_o_proj

# There are 32 layers
params = lora_layer * 32

# So the total size in MB is
size = (params * 4) / 1_000_000

print(f"Total number of params: {params:,} and size {size:,.1f}MB")